In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/nexus-by-djs-nsdc-ultraceuticals'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nexus-by-djs-nsdc-ultraceuticals/sample_submission.csv
/kaggle/input/nexus-by-djs-nsdc-ultraceuticals/train.csv
/kaggle/input/nexus-by-djs-nsdc-ultraceuticals/test.csv


In [2]:
from sklearn.ensemble import RandomForestClassifier
from category_encoders import TargetEncoder
from sklearn.preprocessing import LabelEncoder

In [3]:
df = pd.read_csv('/kaggle/input/nexus-by-djs-nsdc-ultraceuticals/train.csv')
df2 = pd.read_csv('/kaggle/input/nexus-by-djs-nsdc-ultraceuticals/test.csv')
print([str(x) for x in df.iloc[0:0,:]])

['ID', 'TargetID', 'DRUGID', 'DRUGTYPE', 'Drug_high_status', 'DRUGNAME', 'PUBCHCID', 'Disease_of_highest_status', 'Drug_Status', 'UNIPROID', 'TARGNAME', 'GENENAME', 'SYNONYMS', 'FUNCTION', 'BIOCLASS', 'SEQUENCE', 'Disease', 'Accession Number', 'Target_Status']


In [4]:
main_vars18 = ['ID', 'DRUGNAME', 'GENENAME', 'SEQUENCE', 'Disease']
X18_train = df[main_vars18]
y18_train = df['Target_Status']

label_encoder = LabelEncoder()
y18_train = label_encoder.fit_transform(y18_train)  # Ensure you use y18_train directly

X18_test = df2[main_vars18]

target_encoder = TargetEncoder(cols=['DRUGNAME', 'GENENAME', 'SEQUENCE', 'Disease'])
X18_train_encoded = target_encoder.fit_transform(X18_train.drop(columns=['ID']), y18_train)
X18_test_encoded = target_encoder.transform(X18_test.drop(columns=['ID']))

X18_train_encoded['ID'] = X18_train['ID'].values
X18_test_encoded['ID'] = X18_test['ID'].values

model18 = RandomForestClassifier(random_state=42)
model18.fit(X18_train_encoded.drop(columns=['ID']), y18_train)

y18_pred = model18.predict(X18_test_encoded.drop(columns=['ID']))

submission18 = pd.DataFrame({
    'ID': df2['ID'], 
    'Prediction': label_encoder.inverse_transform(y18_pred) 
})

submission18.to_csv('submission18.csv', index=False)